In [ ]:
#Daily Rain NC file conversion to Raster TIF file into a single channel


import numpy as np
from scipy.interpolate import griddata


# Generating Total Rain per year.
def interpolate_zeros_and_nans(array):
    # Get the indices of zero and NaN values
    zero_nan_indices = np.where((array == 0) | np.isnan(array))
    
    # Get the indices of non-zero and non-NaN values
    valid_indices = np.where((array != 0) & ~np.isnan(array))
    
    # Get the corresponding valid values
    valid_values = array[valid_indices]
    
    # If there are no zero or NaN values to interpolate, return the original array
    if len(zero_nan_indices[0]) == 0:
        return array
    
    # Perform griddata interpolation
    interpolated_values = griddata(
        points=list(zip(valid_indices[0], valid_indices[1])),  # Locations of known values
        values=valid_values,                                   # Known values
        xi=list(zip(zero_nan_indices[0], zero_nan_indices[1])), # Locations to interpolate
        method='nearest'                                       # Interpolation method
    )
    
    # Replace zero and NaN values with interpolated values
    array[zero_nan_indices] = interpolated_values
    
    return array

import xarray as xr
import rasterio
from rasterio.transform import from_origin
import numpy as np
import os

def process_nc_file2(nc_file_path):
    # Open the .nc file using xarray
    with xr.open_dataset(nc_file_path) as dataset:
        # Extract monthly rainfall data
        # Extract the first variable name automatically
        var_name = list(dataset.data_vars.keys())[0]
        print(var_name)
        data = dataset[var_name]
        # Calculate the annual total rainfall (sum over time dimension)
        total_rain = data.sum(dim='time')
        
        # Extract spatial dimensions (lat, lon)
        lat = dataset['lat'].values
        lon = dataset['lon'].values
        
        # Calculate resolution from lat/lon data
        lat_res = abs(lat[1] - lat[0])
        lon_res = abs(lon[1] - lon[0])
        
        # Flip the data array along the latitude axis to match the raster orientation
        total_rain = np.flipud(total_rain.values)
        # total_rain = interpolate_zeros_and_nans(total_rain)

        # print(f"Data shape: {data.shape}")
        
        # # Define transform (assuming data is a regular grid)
        # transform = from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])
        
        # Define transform for the raster file
        transform = from_origin(west=lon.min(), north=lat.max(), xsize=lon_res, ysize=lat_res)
        
        # Prepare the output raster file path
        output_file = os.path.splitext(nc_file_path)[0] + '.tif'
        
        # Save the data as a GeoTIFF file
        with rasterio.open(
            output_file, 
            'w', 
            driver='GTiff', 
            height=total_rain.shape[0], 
            width=total_rain.shape[1], 
            count=1, 
            dtype=np.float32, 
            crs='EPSG:4326',  # Assuming data is in WGS84
            transform=transform
        ) as dst:
            dst.write(total_rain.astype(np.float32), 1)
    
    print(f'Saved {output_file}')

# # Call the function with the path to your .nc file
nc_file = r"H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.daily_rain.nc"
process_nc_file2(nc_file)


daily_rain
Saved H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.daily_rain.tif


In [ ]:
def process_nc_file3(nc_file_path):
    # Open the .nc file using xarray
    with xr.open_dataset(nc_file_path) as dataset:
        # Extract monthly rainfall data
        # Extract the first variable name automatically
        var_name = list(dataset.data_vars.keys())[0]
        print(var_name)
        data = dataset[var_name]
        # Calculate the annual mean value of a weather feature (mean over time dimension)
        mean_feature = data.mean(dim='time')
        
        # Extract spatial dimensions (lat, lon)
        lat = dataset['lat'].values
        lon = dataset['lon'].values
        
        # Calculate resolution from lat/lon data
        lat_res = abs(lat[1] - lat[0])
        lon_res = abs(lon[1] - lon[0])
        
        # Flip the data array along the latitude axis to match the raster orientation
        mean_feature = np.flipud(mean_feature.values)
        # mean_feature = interpolate_zeros_and_nans(mean_feature)

        # print(f"Data shape: {data.shape}")
        
        # # Define transform (assuming data is a regular grid)
        # transform = from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])
        
        # Define transform for the raster file
        transform = from_origin(west=lon.min(), north=lat.max(), xsize=lon_res, ysize=lat_res)
        
        # Prepare the output raster file path
        output_file = os.path.splitext(nc_file_path)[0] + '.tif'
        
        # Save the data as a GeoTIFF file
        with rasterio.open(
            output_file, 
            'w', 
            driver='GTiff', 
            height=mean_feature.shape[0], 
            width=mean_feature.shape[1], 
            count=1, 
            dtype=np.float32, 
            crs='EPSG:4326',  # Assuming data is in WGS84
            transform=transform
        ) as dst:
            dst.write(mean_feature.astype(np.float32), 1)
    
    print(f'Saved {output_file}')

In [8]:
# Multiple .nc files transformation
# Directory containing the .nc files
input_directory = r"H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/"

# Process each .nc file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.nc'):
        file_path = os.path.join(input_directory, filename)
        print(file_path)
        process_nc_file3(file_path)


H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_morton_actual.nc
et_morton_actual
Saved H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_morton_actual.tif
H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_morton_potential.nc
et_morton_potential
Saved H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_morton_potential.tif
H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_morton_wet.nc
et_morton_wet
Saved H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_morton_wet.tif
H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_short_crop.nc
et_short_crop
Saved H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_short_crop.tif
H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/2023.et_tall_crop.nc
et_tall_crop
Saved H:/Food Agility Data/Farmanco Dataset Processing/Weather_Data_2023/202